In [1]:
import os; os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
from tensorflow import keras

In [2]:
IMG_SIZE = (224, 224)

train_ds = keras.utils.image_dataset_from_directory(
    'data/cats_dogs/',
    batch_size=16,
    label_mode='binary',
    image_size=IMG_SIZE,
    shuffle=True,
    seed=42,
    validation_split=0.3,
    subset='training')

val_ds = keras.utils.image_dataset_from_directory(
    'data/cats_dogs/',
    batch_size=16,
    label_mode='binary',
    image_size=IMG_SIZE,
    shuffle=True,
    seed=42,
    validation_split=0.3,
    subset='validation')

Found 25000 files belonging to 2 classes.
Using 17500 files for training.
Found 25000 files belonging to 2 classes.
Using 7500 files for validation.


Per prima cosa, proviamo ad utilizzare la rete in **transfer learning**. Per farlo:

* selezioniamo una rete (in questo caso, `MobileNetV3Small`) dal package `applications` di Keras;
* specifichiamo il parametro `weights` ad `imagenet`, in modo da caricare i pesi della rete già addestrata su ImageNet;
* rimuoviamo il layer di classificazione di ImageNet impostando `include_top` a `False`;
* impostiamo l'attributo `trainable` a `False` per evitare di modificare i pesi della rete;
* costruiamo un nuovo modello.

In [3]:
base_model = keras.applications.MobileNetV3Small(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False
)

base_model.trainable = False

model = keras.Sequential()
model.add(keras.layers.Lambda(keras.applications.mobilenet_v3.preprocess_input, input_shape=(224, 224, 3)))
model.add(base_model)
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1))

model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(),
    metrics=[
        keras.metrics.BinaryAccuracy(),
        keras.metrics.Precision(),
        keras.metrics.Recall()
    ]
)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='checkpoints/checkpoints',
        save_weights_only=True,
        monitor='val_binary_accuracy',
        save_best_only=True),
    keras.callbacks.EarlyStopping(
        monitor='val_binary_accuracy',
        min_delta=0.1,
        patience=3,
        restore_best_weights=True),
    keras.callbacks.TensorBoard(log_dir='logs')
]

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=callbacks)

Epoch 1/20
1094/1094 [==============================] - 142s 128ms/step - loss: 1.3498 - binary_accuracy: 0.9085 - precision: 0.9127 - recall: 0.9026 - val_loss: 2.3737 - val_binary_accuracy: 0.8437 - val_precision: 0.9826 - val_recall: 0.7029
Epoch 2/20
1094/1094 [==============================] - 148s 135ms/step - loss: 1.3812 - binary_accuracy: 0.9087 - precision: 0.8978 - recall: 0.9216 - val_loss: 0.9584 - val_binary_accuracy: 0.9369 - val_precision: 0.9592 - val_recall: 0.9139
Epoch 3/20
1094/1094 [==============================] - 135s 123ms/step - loss: 1.0370 - binary_accuracy: 0.9312 - precision: 0.9049 - recall: 0.9630 - val_loss: 1.0394 - val_binary_accuracy: 0.9315 - val_precision: 0.9629 - val_recall: 0.8988
Epoch 4/20
1094/1094 [==============================] - 126s 116ms/step - loss: 0.8708 - binary_accuracy: 0.9427 - precision: 0.9442 - recall: 0.9406 - val_loss: 1.0643 - val_binary_accuracy: 0.9303 - val_precision: 0.9700 - val_recall: 0.8893


Proviamo adesso ad effettuare il **fine tuning**. Per farlo, impostiamo `trainable` a `True`, e compiliamo il modello usando un learning rate molto basso (in questo caso, 0.00001).

In [5]:
base_model.trainable = True

model.compile(
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(1e-5),
    metrics=[
        keras.metrics.BinaryAccuracy(),
        keras.metrics.Precision(),
        keras.metrics.Recall()
    ]
)

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3,
    callbacks=callbacks)

Epoch 1/3
1094/1094 [==============================] - 421s 377ms/step - loss: 0.4656 - binary_accuracy: 0.9358 - precision_2: 0.9383 - recall_2: 0.9323 - val_loss: 0.3857 - val_binary_accuracy: 0.9403 - val_precision_2: 0.9397 - val_recall_2: 0.9422
Epoch 2/3
1094/1094 [==============================] - 446s 408ms/step - loss: 0.3278 - binary_accuracy: 0.9451 - precision_2: 0.9494 - recall_2: 0.9400 - val_loss: 0.3459 - val_binary_accuracy: 0.9453 - val_precision_2: 0.9382 - val_recall_2: 0.9546
Epoch 3/3
1094/1094 [==============================] - 403s 369ms/step - loss: 0.2893 - binary_accuracy: 0.9502 - precision_2: 0.9536 - recall_2: 0.9459 - val_loss: 0.3079 - val_binary_accuracy: 0.9483 - val_precision_2: 0.9436 - val_recall_2: 0.9546
